# General model for solving biot equation
This notebook contains a method for setting up and solving biot's equation:
\begin{align}
\nabla\cdot\frac{1}{2}D(\nabla u + (\nabla u)^T) - \alpha \nabla p = F \\
\frac{\partial}{\partial t} (\beta p + \alpha \nabla \cdot u ) - \nabla \cdot \nabla p = f
\end{align}
with boundary conditions on $\partial\Omega_d$ and $\partial\Omega_n$:
\begin{align}
p=p_b, \quad\qquad\qquad\qquad -K\nabla\cdot \boldsymbol{n} = v_b \\
\boldsymbol{u}= \boldsymbol{u}_b, \qquad\quad\frac{1}{2}D(\nabla u + (\nabla u)^T)\cdot\boldsymbol{n} = t_b
\end{align}

In the above, the primary variables are the displacements $\boldsymbol{u}$ and pressures $p$. The parameters are the scalar and vector source terms $F$ and $f$, the permeability tensor $K$, the stiffness matrix $D$, the Biot coefficient $\alpha$, the fluid compressibility $\beta$. $p_b$ is the pressure at the boundary with dirichlet condition, $v_b$ is the flux at the neumann boundary, $\boldsymbol{u}_b$ is the displacement at the dirichlet boundary, $t_b$ is the traction at the neumann boundary.

In [ ]:
import time

import numpy as np
import scipy.sparse as sps
import porepy as pp

from porepy.utils.derived_discretizations import implicit_euler as IE_discretizations

## Parent class to the biot equation
Since the pure mechanics problem essentially is one of the involved equations in the biot equations, we can use the pure mechanics class as parent to the Biot class.

In [ ]:
class Mechanics:
    """ Class for setting up a pure mechanics problem with no fractures.
    
    The following attributes are assigned to self:
    Method: __init__
        mesh_args (dict): Mesh sizes
        folder_name (str): Name of folder for visualization
        displacement_variable (str): Name of variable for displacement
        mechanics_parameter_keyword (str): Parameter keyword for mechanics problems
    Method: create_grid
        box (dict): Bounding box of domain.
        gb (pp.GridBucket): The grid bucket.
        Nd (int): Dimension of the domain.
    Method: rock_parameters
        lam (np.ndarray): lambda, Lamé parameter
        mu (np.ndarray): Lamé parameter
        
    The following attributes in self may be updated:
    Method: insert_heterogenity
        lam (np.ndarray)
        mu (np.ndarray)
        
    """

    def __init__(self, mesh_args, folder_name):
        """ Initialize the mechanics class.
        
        Parameters
        mesh_args (dict): Containing mesh sizes.
                For now, this should be e.g. {'n': 5}. 
                We use cartesian meshes.
        folder_name (str): Name of storage folder for visualization.
        
        """
        from porepy.geometry.geometry_property_checks import point_in_polygon
        self.mesh_args = mesh_args
        self.folder_name = folder_name
        
        # Variable
        self.displacement_variable = "u"
        
        # Keyword
        self.mechanics_parameter_keyword = "mechanics"
        
        
    def create_grid(self):
        """ Create a 2D domain without fractures.
        
        The method requires the following attribute:
            mesh_args (dict): Containing the mesh sizes
        
        The following attributes are assigned to self:
            box (dict): Bounding box of domain.
            gb (pp.GridBucket): The grid bucket.
            Nd (int): Dimension of the domain.
        
        """
        n = self.mesh_args['n']
        nx = np.array((n,n))
        self.box = {'xmin': 0, 'ymin': 0, 'xmax': 1, 'ymax': 1}
        
        #g = pp.CartGrid([n,n], physdims=[1,1])
        #g.compute_geometry()
        gb = pp.meshing.cart_grid([], nx, physdims=[1,1])
        gb.compute_geometry()
        gb.assign_node_ordering()
        
        self.gb = gb
        self.Nd = gb.dim_max()
        
    def domain_boundary_sides(self, g):
        """ 
        Obtain indices for faces on each side of the domain boundary.
        
        """
        tol = 1e-10
        box = self.box
        east = g.face_centers[0] > box["xmax"] - tol
        west = g.face_centers[0] < box["xmin"] + tol
        north = g.face_centers[1] > box["ymax"] - tol
        south = g.face_centers[1] < box["ymin"] + tol
        if self.Nd == 2:
            top = np.zeros(g.num_faces, dtype=bool)
            bottom = top.copy()
        else:
            top = g.face_centers[2] > box["zmax"] - tol
            bottom = g.face_centers[2] < box["zmin"] + tol
        
        all_bf = g.get_boundary_faces()
        return all_bf, east, west, north, south
        
    def bc_type(self, g):
        all_bf, *_ = self.domain_boundary_sides(g)
        bc = pp.BoundaryConditionVectorial(g, all_bf, "dir")
        return bc
    
    def bc_values(self, g):
        values = np.zeros((self.Nd, g.num_faces))
        values = values.ravel('F')
        return values

    def source(self, g):
        return np.zeros(g.dim * g.num_cells)
    
    def rock_parameters(self, g):
        """ Assemble the stress-strain tensor.
        
        The following attributes are assigned to self:
            lam (np.ndarray, (g.num_cells,)): lambda parameter
            mu (np.ndarray, (g.num_cells,)): mu parameter.

        """
        lam = np.ones(g.num_cells)
        mu = np.ones(g.num_cells)
        C = pp.FourthOrderTensor(mu, lam)
        self.lam = lam
        self.mu = mu
        return C
    
    def set_parameters(self):
        """ 
        Set default parameters to the matrix

        """
        for g, d in self.gb:
            if g.dim == self.Nd:
                
                # Rock parameters
                C = self.rock_parameters(g)
                
                # Boundary conditions and source value
                bc = self.bc_type(g)
                bc_val = self.bc_values(g)
                f = self.source(g)

                specified_parameters = {'fourth_order_tensor': C, 'source': f, 
                                        'bc': bc, 'bc_values': bc_val}
                pp.initialize_default_data(g, d, self.mechanics_parameter_keyword, 
                                           specified_parameters)
        
    def insert_heterogenity(self, physdims, center, mu, lam, tol=1e-8):
        """ Insert a heterogenity to the domain.

        A heterogenity in this sense means a rectangle with
        a different mu and/or lambda (e.g. different C) than
        the surrounding matrix.
        Assumes that parameters are already been set. Assumes existense
        of attributes self.lam and self.mu

        Parameters:
        physdims (np.ndarray, (2,) ): Physical dimensions of rectangle
        center (np.ndarray, (2,)): Coordinates of center of rectangle
        mu (float): Value of mu in the rectangle.
        lam (float): Value of lambda in the rectangle.
        tol (float, Default: 1e-8): Tolerance of rectangle bounds.

        The following attributes in self are updated:
            mu (np.ndarray, (g.num_cells,)): mu parameter
            lam (np.ndarray, (g.num_cells,)): Lambda parameter

        In addition, the parameter dictionary for the domain of highest dimension
        (we assume no fracs) has its entry 'fourth_order_tensor' updated

        """
        from porepy.geometry.geometry_property_checks import point_in_polygon
        g = self.gb.grids_of_dimension(self.Nd)[0]
        data = self.gb.node_props(g)
        R = RectangleByCenter(physdims, center)

        # This code assumes 2D.
        nds = np.copy(g.cell_centers)
        nds[2] = np.array(range(g.num_cells)) # Use z-coordinate as index.
        rect_nodes = nds[2, point_in_polygon(R.nodes, nds[[0,1],:], tol=tol)].astype(int)

        self.mu[rect_nodes] = mu
        self.lam[rect_nodes] = lam
        C = pp.FourthOrderTensor(self.mu, self.lam)

        data[pp.PARAMETERS]['fourth_order_tensor'] = C
        
    def assign_variables(self):
        """ 
        Assign variables to the node of the grid bucket.

        """
        gb = self.gb
        for g, d in gb:
            if g.dim == self.Nd:
                d[pp.PRIMARY_VARIABLES] = {
                    self.displacement_variable: {"cells": self.Nd}
                }
            else:
                d[pp.PRIMARY_VARIABLES] = {}
        
        for e, d in gb.edges():
            
            if e[0].dim == self.Nd:
                pass
            
            d[pp.PRIMARY_VARIABLES] = {}
    
    def assign_discretization(self):
        """ 
        Assign discretization to the node of the grid bucket
        
        """
        # We solve the highest dimension problem using mpsa.
        # There should be no other dimensions present.
        Nd = self.Nd
        gb = self.gb
        mpsa = pp.Mpsa(self.mechanics_parameter_keyword)
        
        for g, d in gb:
            if g.dim == Nd:
                d[pp.DISCRETIZATION] = {
                    self.displacement_variable: {'mpsa': mpsa}}
        
        # Assume no edges, so we don't loop over them.
        
    def initial_condition(self):
        """ Set initial guess for the variable.
        
        Set displacement to zero in the matrix.
        """
        
        for g, d in self.gb:
            if g.dim == self.Nd:
                state = {
                    self.displacement_variable: np.zeros(g.num_cells * self.Nd)
                }
                
        pp.set_state(d, state)
        

In [ ]:
class Biot(Mechanics):
    
    def __init__(self, mesh_args, folder_name, params=None, **kwargs):
        """ Assemble and run the Biot model.
        
        Parameters:
        mesh_args (dict): Arguments used for meshing
        folder_name (str): Name of folder for visualization
        params (dict): Various parameters. For instance (key:value):
            'linear_solver': 'direct', 'pyamg',
        
        """
        super().__init__(mesh_args, folder_name)
        
        # Variables
        self.displacement_variable = 'u'
        self.scalar_variable = 'p'
        
        self.mechanics_parameter_key = 'mechanics'
        self.scalar_parameter_key = 'flow'
        
        # Scaling coefficients
        self.scalar_scale = 1
        self.length_scale = 1
        
        # Time stepping
        self.time_step = kwargs.get('time_step', 1)
        
    def bc_type_mechanics(self, g):
        """ Use parent class for bc"""
        return super().bc_type(g)
    
    def bc_type_scalar(self, g):
        all_b, *_ = self.domain_boundary_sides(g)
        return pp.BoundaryCondition(g, all_bf, 'dir')
    
    def bc_values_mechanics(self, g):
        return super().bc_values(g)
    
    def bc_values_scalar(self, g):
        return np.zeros(g.num_faces)
    
    def source_mechanics(self, g):
        return super().source(g)

    def source_scalar(self, g):
        return np.zeros(g.num_cells)
    
    def biot_alpha(self):
        return 1
    
    def set_parameters(self):
        """
        Set the parameters for the model
        """
        self.set_scalar_parameters()
        self.set_mechanics_parameters()
    
    def rock_parameters(self, g):
        """ Assemble the stress-strain tensor.
        
        Overwrites parent to account for scales.
        
        """
        lam = np.ones(g.num_cells) / self.scalar_scale
        mu = np.ones(g.num_cells) / self.scalar_scale
        C = pp.FourthOrderTensor(mu, lam)
        self.lam = lam
        self.mu = mu
        return C
        
    def set_mechanics_parameters(self):
        """ Set the mechanics related parameters for the model"""
        gb = self.gb
        
        for g, d in gb:
            if g.dim == self.Nd:
                
                # Rock parameters
                C = self.rock_parameters(g)
                
                # Boundary conditions and source term
                bc = self.bc_type_mechanics(g)
                bc_val = self.bc_values_mechanics(g)
                source = self.source_mechanics(g)
                
                specified_parameters = {
                    'fourth_order_tensor': C,
                    'bc': bc,
                    'bc_values': bc_val,
                    'source': source,
                    'time_step': self.time_step,
                    'biot_alpha': self.biot_alpha(),
                }
                
                pp.initialize_default_data(g, d,
                                           self.mechanics_parameter_key,
                                           specified_parameters,
                                          )
                
    def set_scalar_parameters(self):
        gb = self.gb
        
        tensor_scale = self.scalar_scale / self.length_scale ** 2
        kappa = 1 * tensor_scale
        mass_weight = 1
        alpha = self.biot_alpha()
        for g, d in gb:
            bc = self.bc_type_scalar(g)
            bc_values = self.bc_values_scalar(g)
            source = self.source_scalar(g)
            
            diffusivity = pp.SecondOrderTensor(
                kappa * np.ones(g.num_cells)
            )
            
            specified_parameters = {
                'bc': bc,
                'bc_values': bc_values,
                'mass_weight': mass_weight,
                'biot_alpha': alpha,
                'source': source,
                'second_order_tensor': diffusivity,
                'time_step': self.time_step,
            }
            
            pp.initialize_default_data(
                g,
                d,
                self.scalar_parameter_key,
                specified_parameters,
            )
    
    def assign_discretization(self):
        """
        Assign discretizations to the grid in the grid bucket.
        
        """
        # Shorthand
        key_s, key_m = self.scalar_parameter_key, self.mechanics_parameter_key
        var_s, var_d = self.scalar_variable, self.displacement_variable
        
        # Define discretization
        # Linear elasticity 
        mpsa = pp.Mpsa(key_m)
        # Scalar discretizations
        diff_disc_s = IE_discretizations.ImplicitMpfa(key_s)
        mass_disc_s = IE_discretizations.ImplicitMassMatrix(key_s, var_s)
        source_disc_s = pp.ScalarSource(key_s)
        # Coupling discretizations
        div_u_disc = pp.DivU(
            key_m,
            key_s,
            variable=var_d,
        )
        grad_p_disc = pp.GradP(key_m)
        stabilization_disc_s = pp.BiotStabilization(key_s, var_s)
        
        # Assign discretizations
        for g, d in self.gb:
            if g.dim == self.Nd:
                d[pp.DISCRETIZATION] = {
                    var_d: {'mpsa': mpsa},
                    var_s: {
                        'diffusion': diff_disc_s,
                        'mass': mass_disc_s,
                        'stabilization': stabilization_disc_s,
                        'source': source_disc_s,
                    },
                    var_d + '_' + var_s: {'grad_p': grad_p_disc},
                    var_s + '_' + var_d: {'div_u': div_u_disc},
                }
    
    def assign_variables(self):
        """
        Assign primary variables to the grid in the grid bucket.
        """
        for g, d in self.gb:
            if g.dim == self.Nd:
                d[pp.PRIMARY_VARIABLES] = {
                    self.displacement_variable: {'cells': self.Nd},
                    self.scalar_variable: {'cells': 1},
                }
    
    def discretize_biot(self, gb):
        """ To save computational time, the full Biot
        (without contact mechanics) is discretized once.
        This avoids computing the same term multiple times.
        """
        g = gb.grids_of_dimension(gb.dim_max())[0]
        d = gb.node_props(g)
        biot = pp.Biot(
            mechanics_keyword=self.mechanics_parameter_key,
            flow_keyword=self.scalar_parameter_key,
            vector_variable=self.displacement_variable,
            scalar_variable=self.scalar_variable,
        )
        biot.discretize(g, d)
    
    def initial_condition(self):
        """
        Initial guess for the Newton iteration, scalar variable and 
        bc_values (for tme discretization).
        """
        super().initial_condition()
        
        for g, d in self.gb:
            # Initial scalar value
            initial_scalar_value = np.zeros(g.num_cells)
            d[pp.STATE].update({self.scalar_variable: initial_scalar_value})
            if g.dim == self.Nd:
                bc_values = d[pp.PARAMETERS][self.mechanics_parameter_key]['bc_values']
                mech_dict = {'bc_values': bc_values}
                d[pp.STATE].update({self.mechanics_parameter_key: mech_dict})
                
    
    def export_step(self):
        pass
    
    def export_pvd(self):
        pass
    
    def prepare_simulation(self):
        """ Is run prior to a time-stepping scheme. Initializes
        discretizations, linear solvers, etc.
        """
        self.create_grid()
        self.set_parameters()
        self.initial_condition()
        
        self.assign_variables()
        self.assign_discretization()
        self.discretize()
        self.initialize_linear_solver()
        
    def discretize(self):
        """ 
        Discretize all terms
        """
        if not hasattr(self, "assembler"):
            self.assembler = pp.Assembler(self.gb)
            
        g_max = self.gb.grids_of_dimension(self.Nd)[0]
        
        tic = time.time()
        
        # Next, discretization, which is a bit tricky. See
        # contact_mechanics_biot_model for details.
        # First, discretize the biot class
        self.discretize_biot(self.gb)
        
        # Next, discretize the term on the matrix grid not covered by
        # Biot discretization, i.e. the source term
        self.assembler.discretize(grid=g_max, term_filter=['source'])
        
    def before_newton_loop(self):
        """ Will be run before entering a Newton loop. Discretize time-dependent quantities etc.
        """
        # self.discretize()
    
    def after_newton_iteration(self, solution):
        self.update_state(solution)
        
    def after_newton_convergence(self, solution):
        self.assembler.distribute_variable(solution)
    
    def after_newton_divergence(self):
        raise ValueError("Newton iterations did not converge")
        
    def initialize_linear_solver(self):
        """ Choose which solver to use.
        See contact_mechanics_biot_model for details.
        Here, we only use direct solver, hence pass this method.
        
        """
        self.linear_solver = 'direct'
            
    def assemble_and_solve_linear_system(self, tol):
        
        A, b = self.assembler.assemble_matrix_rhs()
        return sps.linalg.spsolve(A, b)
        

# Test the biot code


In [ ]:
class BiotExample(Biot):
    """ Set up a particular example of the Biot"""
    
    def __init__(self):
        mesh_args = {'n': 5}
        folder_name = "biot"
        super().__init__(mesh_args, folder_name)
        
    def bc_type_mechanics(self, g):
        
        bound_faces = g.get_all_boundary_faces()
        bound_mech = pp.BoundaryConditionVectorial(
            g, bound_faces.ravel("F"), ["dir"] * bound_faces.size
        )
        
    def bc_type_scalar(self, g):
        
        bound_faces = g.get_all_boundary_faces()
        bound_flow = pp.BoundaryCondition(
            g, bound_faces.ravel("F"), ["dir"] * bound_faces.size
        )
        return bound_flow
    
    def bc_values_scalar(self, g):
        boundary_values_flow = np.zeros(g.num_faces)
        boundary_values_flow[0] = 1
        return boundary_values_flow
    
    def update_state(self, solution):
        """ Update states"""
        displacement = data[pp.STATE][variable_m]
        pressure = data[pp.STATE][variable_f]
        pp.set_state(data, {variable_m: displacement, variable_f: pressure})
    
    def run_biot(self):
        """ Solve the biot equation on a 2D non-fractured domain."""
        # Prepare simulation
        tol = 1e-10
        self.prepare_simulation()
        
        time_steps = np.arange(3)
        stored_pressures = []
        stored_displacements = []
        for _ in time_steps:
            # Solve
            x = self.assemble_and_solve_linear_system(tol)
            # Distribute variables
            self.after_newton_convergence(x)
            stored_pressures.append(pressure)
            stored_displacements.append(displacement)
            
        return stored_pressures, stored_displacements
        

# Run biot model 

In [ ]:
b = BiotExample()

In [ ]:
p, d = b.run_biot()